In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
from gensim.models import FastText
import numpy as np

data = pd.read_csv('urdu-sentiment-corpus-v1.tsv', delimiter='\t')
data.columns = ['Tweet', 'Class']
data['Class'] = data['Class'].map({'P': 1, 'N': 0})
data = data.dropna()
data

X = data['Tweet'].values
y = data['Class'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [2]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_len = max([len(seq) for seq in X_train + X_test])
vocab_size = len(tokenizer.word_index) + 1

X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

word2vec_model = Word2Vec(sentences=data['Tweet'], vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.save("word2vec_urdu.model")


In [3]:
def create_bilstm_model(embedding_matrix):
    model = Sequential()
    model.add(Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], input_length=max_len, trainable=False))
    model.add(Bidirectional(LSTM(100, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(100, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(100)))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
def evaluate_classifier(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return acc, precision, recall, f1

results = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1-Score'])

Word2Vec

In [5]:
word2vec_model = Word2Vec.load("word2vec_urdu.model")
word_vectors = word2vec_model.wv
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

bilstm_model = create_bilstm_model(embedding_matrix)
bilstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1)

acc, precision, recall, f1 = evaluate_classifier(bilstm_model, X_test, y_test)

results.loc['BiLSTM_Word2Vec'] = [acc, precision, recall, f1]

print(results)


Epoch 1/10
11/11 [==============================] - 17s 533ms/step - loss: 0.6947 - accuracy: 0.5152 - val_loss: 0.6960 - val_accuracy: 0.3784
Epoch 2/10
11/11 [==============================] - 3s 272ms/step - loss: 0.6944 - accuracy: 0.4955 - val_loss: 0.6977 - val_accuracy: 0.4730
Epoch 3/10
11/11 [==============================] - 3s 291ms/step - loss: 0.6926 - accuracy: 0.5258 - val_loss: 0.7019 - val_accuracy: 0.4865
Epoch 4/10
11/11 [==============================] - 3s 292ms/step - loss: 0.6932 - accuracy: 0.4758 - val_loss: 0.7018 - val_accuracy: 0.4730
Epoch 5/10
11/11 [==============================] - 3s 302ms/step - loss: 0.6916 - accuracy: 0.5167 - val_loss: 0.7062 - val_accuracy: 0.4865
Epoch 6/10
11/11 [==============================] - 3s 304ms/step - loss: 0.6915 - accuracy: 0.5045 - val_loss: 0.6998 - val_accuracy: 0.4730
Epoch 7/10
11/11 [==============================] - 3s 297ms/step - loss: 0.6898 - accuracy: 0.5182 - val_loss: 0.7058 - val_accuracy: 0.4730
Epoch

Glove

In [6]:
embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

bilstm_model = create_bilstm_model(embedding_matrix)
bilstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1)
acc, precision, recall, f1 = evaluate_classifier(bilstm_model, X_test, y_test)
results.loc['BiLSTM_Glove'] = [acc, precision, recall, f1]

print(results)

Epoch 1/10
11/11 [==============================] - 18s 543ms/step - loss: 0.6954 - accuracy: 0.4742 - val_loss: 0.6935 - val_accuracy: 0.5270
Epoch 2/10
11/11 [==============================] - 3s 268ms/step - loss: 0.6905 - accuracy: 0.5348 - val_loss: 0.6974 - val_accuracy: 0.5135
Epoch 3/10
11/11 [==============================] - 3s 280ms/step - loss: 0.6854 - accuracy: 0.5485 - val_loss: 0.7173 - val_accuracy: 0.5135
Epoch 4/10
11/11 [==============================] - 3s 296ms/step - loss: 0.6850 - accuracy: 0.5515 - val_loss: 0.6997 - val_accuracy: 0.5270
Epoch 5/10
11/11 [==============================] - 3s 295ms/step - loss: 0.6806 - accuracy: 0.5515 - val_loss: 0.7023 - val_accuracy: 0.4459
Epoch 6/10
11/11 [==============================] - 3s 303ms/step - loss: 0.6751 - accuracy: 0.5470 - val_loss: 0.6959 - val_accuracy: 0.5405
Epoch 7/10
11/11 [==============================] - 3s 309ms/step - loss: 0.6705 - accuracy: 0.5621 - val_loss: 0.7109 - val_accuracy: 0.5270
Epoch

fastText

In [8]:
X_sentences = [sentence.split() for sentence in X]
fasttext_model = FastText(X_sentences, vector_size=32, min_count=1)
fasttext_dict = {word: fasttext_model.wv[word] for word in fasttext_model.wv.index_to_key}
embedding_matrix = np.zeros((5000, 32))
for word, i in tokenizer.word_index.items():
    embedding_vector = fasttext_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

bilstm_model = create_bilstm_model(embedding_matrix)
bilstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1)

acc, precision, recall, f1 = evaluate_classifier(bilstm_model, X_test, y_test)

results.loc['BiLSTM_fastText'] = [acc, precision, recall, f1]

print(results)

Epoch 1/10
11/11 [==============================] - 24s 1s/step - loss: 0.6955 - accuracy: 0.4833 - val_loss: 0.6884 - val_accuracy: 0.5541
Epoch 2/10
11/11 [==============================] - 9s 782ms/step - loss: 0.6937 - accuracy: 0.5076 - val_loss: 0.6900 - val_accuracy: 0.5541
Epoch 3/10
11/11 [==============================] - 9s 786ms/step - loss: 0.6935 - accuracy: 0.4955 - val_loss: 0.6913 - val_accuracy: 0.5541
Epoch 4/10
11/11 [==============================] - 9s 799ms/step - loss: 0.6925 - accuracy: 0.5091 - val_loss: 0.6890 - val_accuracy: 0.5541
Epoch 5/10
11/11 [==============================] - 9s 778ms/step - loss: 0.6944 - accuracy: 0.5045 - val_loss: 0.6904 - val_accuracy: 0.5541
Epoch 6/10
11/11 [==============================] - 9s 782ms/step - loss: 0.6926 - accuracy: 0.5212 - val_loss: 0.6897 - val_accuracy: 0.5541
Epoch 7/10
11/11 [==============================] - 9s 776ms/step - loss: 0.6942 - accuracy: 0.5045 - val_loss: 0.6895 - val_accuracy: 0.5541
Epoch 8/

C:\Users\DURRANI\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
